In [1]:
# Load the Pre-trained VGG16 Model
# import VGG16 from keras.applications and load the model with ImageNet weights, excluding the top (fully connected) layers.

from tensorflow.keras.applications import VGG16
from tensorflow.image import resize
from cv2 import imread, imshow
import numpy as np
import os

# Load the VGG16 model, pre-trained weights, exclude top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [2]:
# Add Custom Layers
# Freeze the layers of the base model to prevent them from being updated during training, then add your custom layers on top for the classification task.
from tensorflow.keras import layers, models

# Freeze the base model
base_model.trainable = False

# Create a custom head for our network
model = models.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(2, activation='softmax')  # Assuming two classes: Cat and Dog
])

In [3]:
# Compile and Train the Model
# Compile the model, specifying the loss function, optimizer, and metrics. Then, train the model with your dataset.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
train_images, test_images, train_labels, test_labels = [], [], [], []
DIR = 'dataset/'

def split_train_test():
    for root, dir, files in os.walk('./dataset'):
        for file in files:
            img = imread(DIR + file)
            processed = resize(
                np.expand_dims(img, axis=0), [224, 224]
            )
            cat_or_dog = 1 if file.startswith('cat') else 0
            # There are 12500 cats and dogs (indexed 0 to 12499) in the data set.
            # To make an 80%:20% train-test split, 10000 cats and 10000 dogs need to go into the train set.
            # All other cat and dog images go into the test set.
            if int(file[file.index('.')+1:file.rindex('.')]) < 10000:
                train_images.append(processed)
                train_labels.append(cat_or_dog)
            else:
                test_images.append(processed)
                test_labels.append(cat_or_dog)

split_train_test()

In [6]:
print(train_labels[1])
print(train_images[1])
train_labels_np = np.asarray(train_labels)
train_images_np = np.array(train_images,dtype=np.float32).reshape((20000,224,224,3))
print(train_labels_np[1])
print(train_images_np[1])

1
tf.Tensor(
[[[[ 40.125     44.125     39.125   ]
   [ 41.58817   44.061382  39.570312]
   [ 43.848213  44.848213  40.848213]
   ...
   [180.87604  206.7868   210.63501 ]
   [169.12376  201.10594  204.10594 ]
   [163.03015  197.69086  200.69086 ]]

  [[ 41.        45.        40.      ]
   [ 42.017857  44.49107   40.      ]
   [ 43.530132  44.530132  40.530132]
   ...
   [174.55463  200.35153  204.25667 ]
   [165.6651   196.71312  200.27228 ]
   [160.67636  194.21207  197.96207 ]]

  [[ 40.268974  44.268974  39.268974]
   [ 40.767857  43.24107   38.75    ]
   [ 41.219868  42.219868  38.219868]
   ...
   [168.77898  192.79578  197.3259  ]
   [164.99554  194.48663  198.97772 ]
   [165.66518  197.32588  202.32588 ]]

  ...

  [[ 28.785715  27.785715  29.785715]
   [ 25.725447  24.725447  26.725447]
   [ 21.830357  20.830357  22.830357]
   ...
   [ 25.214233  31.214233  44.214233]
   [ 21.107178  27.107178  39.125   ]
   [ 32.815792  38.815792  49.815792]]

  [[ 30.660715  29.660715  31.66

In [9]:
# Assuming `train_images`, `train_labels` is your training dataset
model.fit(train_images_np, train_labels_np, epochs=1, validation_data=(test_images,test_labels))

MemoryError: Unable to allocate 11.2 GiB for an array with shape (20000, 224, 224, 3) and data type float32

In [ ]:
# Evaluate the Model
# After training, evaluate your model's performance on a separate test dataset to see how well it performs.
# Assuming `test_images`, `test_labels` is your testing dataset
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
# Save and Load Your Model
# Finally, save your trained model for later use and load it whenever you need to classify new images.
model.save('cats_vs_dogs_model.h5')


# To load the model:
from tensorflow.keras.models import load_model

model = load_model('cats_vs_dogs_model.h5')